# Introduction

This script compares images using image similarity metrics known as Structural Similarity (CW-SSIM) index and the Complex Wavelet SSIM (CW-SSIM).  It shows how varying an input, which specifies the dynamic range of the images, can change the basic SSIM measure but not the CW-SSIM measure. 

The CW-SSIM is derived from the basic SSIM index, which compares images based on their local intensity, local contrast, and local structure.  However, the basic SSIM index is based on spatial information and so is very sensitive to small image scaling, rotations, or translations.  By contrast, the CW-SSIM is based on wavelet transform coefficients, where such image transformations are just a constant phase shift.  This makes it more robust to scaling, rotation, and translation errors, while maintaining sensitivity to structural image differences.

The following references give more information about CW-SSIM and image similarity metrics in general.

Wang, Z., and A. C. Bovik. 2009. “Mean Squared Error: Love It or Leave It? A New Look at Signal Fidelity Measures.” IEEE Signal Processing Magazine 26 (1):98–117. https://doi.org/10.1109/MSP.2008.930649.

Wang, Zhou, and E. P. Simoncelli. 2005. “Translation Insensitive Image Similarity in Complex Wavelet Domain.” In Proceedings. (ICASSP ’05). IEEE International Conference on Acoustics, Speech, and Signal Processing, 2005., 2:573–76. https://doi.org/10.1109/ICASSP.2005.1415469.


This implementation was done using the pyssim package: https://github.com/jterrace/pyssim


## Code dependencies

In [56]:
import numpy as np

import matplotlib.pyplot as plt
from PIL import Image, ImageOps

from ssim import SSIM
from ssim.utils import get_gaussian_kernel

gaussian_kernel_sigma = 1.5
gaussian_kernel_width = 11
gaussian_kernel_1d = get_gaussian_kernel(gaussian_kernel_width, gaussian_kernel_sigma)

## Open images

In [43]:
pol = Image.open('F:/Box Sync/Research/Polarimetry/RAnalysis/WP4/maskedFixed-weka-1xRes.tif')
mmp = Image.open('F:/Box Sync/Research/Polarimetry/RAnalysis/WP4/JupyterOutput/jupyterAffine_mean_4Pyramid_gradDes_lr1_conMin-1e-6_.tif')
polNW = Image.open('F:/Box Sync/Research/Polarimetry/RAnalysis/WP4/fixed.tif')


# Calculate the SSIM and CW-SSIM

The following code calculates the SSIM and CW-SSIM for 3 different scenarios, in pairs.  

The pair is comparing the SSIM based on a masked Polscope image, where the background is cropped out, to the SSIM based on a Polscope image without the mask.

The three scenarios are adjusting the dynamic range of the algorithm input. This substantially changes the results.

## SSIM

In [65]:
ssim_pol_mmp_drDef = SSIM(pol, gaussian_kernel_1d).ssim_value(mmp)
ssim_polNW_mmp_drDef = SSIM(polNW,gaussian_kernel_1d).ssim_value(mmp)

ssim_pol_mmp = SSIM(pol, gaussian_kernel_1d,l=np.max(pol)).ssim_value(mmp)
ssim_polNW_mmp = SSIM(polNW,gaussian_kernel_1d,l=np.max(pol)).ssim_value(mmp)

ssim_pol_mmp_drOne = SSIM(pol, gaussian_kernel_1d,l=1).ssim_value(mmp)
ssim_polNW_mmp_drOne = SSIM(polNW,gaussian_kernel_1d,l=1).ssim_value(mmp)


print("Dynamic Range = 255 (Code default)")
print("SSIM of MMP to Pol %.4f" % ssim_pol_mmp_drDef)
print("SSIM of MMP to No mask Pol %.4f" % ssim_polNW_mmp_drDef)

print("\nDynamic Range = Maximum value in Polscope Matrix")
print("SSIM of MMP to Pol %.4f" % ssim_pol_mmp)
print("SSIM of MMP to No mask Pol %.4f" % ssim_polNW_mmp)

print("\nDynamic Range = 1 (Matlabs default)")
print("SSIM of MMP to Pol %.4f" % ssim_pol_mmp_drOne)
print("SSIM of MMP to No mask Pol %.4f" % ssim_polNW_mmp_drOne)





Dynamic Range = 255 (Code default)
SSIM of MMP to Pol 0.9392
SSIM of MMP to No mask Pol 0.9288

Dynamic Range = Maximum value in Polscope Matrix
SSIM of MMP to Pol 0.6058
SSIM of MMP to No mask Pol 0.5473

Dynamic Range = 1 (Matlabs default)
SSIM of MMP to Pol 0.4107
SSIM of MMP to No mask Pol 0.2720


As you can see, putting the dynamic range = the maximum value of the polscope matrix, gives a reasonable metric value - as if it were a blurred image.  

However, do note that this value is slightly different than what Matlab produces.  The way they define the dynamic range probably differs.

## CW-SSIM

In [66]:
cw_ssim_pol_mmp_drDef = SSIM(pol).cw_ssim_value(mmp)
cw_ssim_polNW_mmp_drDef = SSIM(polNW).cw_ssim_value(mmp)

cw_ssim_pol_mmp = SSIM(pol,l=np.max(pol)).cw_ssim_value(mmp)
cw_ssim_polNW_mmp = SSIM(polNW,l=np.max(pol)).cw_ssim_value(mmp)

cw_ssim_pol_mmp_drOne = SSIM(pol,l=1).cw_ssim_value(mmp)
cw_ssim_polNW_mmp_drOne = SSIM(polNW,l=1).cw_ssim_value(mmp)


print("Dynamic Range = 255 (Code default)")
print("CW-SSIM of MMP to Pol %.4f" % cw_ssim_pol_mmp_drDef)
print("CW-SSIM of MMP to No mask Pol %.4f" % cw_ssim_polNW_mmp_drDef)

print("\nDynamic Range = Maximum value in Polscope Matrix")
print("CW-SSIM of MMP to Pol %.4f" % cw_ssim_pol_mmp)
print("CW-SSIM of MMP to No mask Pol %.4f" % cw_ssim_polNW_mmp)

print("\nDynamic Range = 1 (Matlabs default)")
print("CW-SSIM of MMP to Pol %.4f" % cw_ssim_pol_mmp_drOne)
print("CW-SSIM of MMP to No mask Pol %.4f" % cw_ssim_polNW_mmp_drOne)


Dynamic Range = 255 (Code default)
CW-SSIM of MMP to Pol 0.5865
CW-SSIM of MMP to No mask Pol 0.4194

Dynamic Range = Maximum value in Polscope Matrix
CW-SSIM of MMP to Pol 0.5865
CW-SSIM of MMP to No mask Pol 0.4194

Dynamic Range = 1 (Matlabs default)
CW-SSIM of MMP to Pol 0.5865
CW-SSIM of MMP to No mask Pol 0.4194


Note that changing the dynamic range has no effect on the results.  This is very advantageous, as it means one less variable for the user to adjust.  This makes it more robust and probably more accurate.